In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output

def read_dataset(template, start_idx, end_idx):
    frames = [ pd.read_json(f) for f in [template.format(i) for i in range(start_idx, end_idx+1)] ]
    return pd.concat(frames, ignore_index = True)

dftrain = read_dataset("datasets/dataset_initial/dataset_{:02}.json", 0, 13)
y_train = dftrain.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)

dfeval = read_dataset("datasets/dataset_initial/dataset_{:02}.json", 13, 17)
y_eval = dfeval.pop('round_winner').map(lambda s: 0 if s == "CT" else 1)
print(dftrain.head())

        map  patch_version    map_crc current_score round_status  \
0  de_dust2          13737  702125823        [0, 0]   FreezeTime   
1  de_dust2          13737  702125823        [0, 0]   FreezeTime   
2  de_dust2          13737  702125823        [0, 0]       Normal   
3  de_dust2          13737  702125823        [0, 0]       Normal   
4  de_dust2          13737  702125823        [1, 0]   FreezeTime   

   round_status_time_left                                      alive_players  \
0                20.00000  [{'health': 100, 'armor': 0, 'has_helmet': Fal...   
1                 1.03125  [{'health': 100, 'armor': 100, 'has_helmet': F...   
2                96.03125  [{'health': 100, 'armor': 100, 'has_helmet': F...   
3                76.03125  [{'health': 100, 'armor': 100, 'has_helmet': F...   
4                19.96875  [{'health': 100, 'armor': 100, 'has_helmet': F...   

                                       active_smokes active_molotovs  \
0                                     

In [6]:
CATEGORICAL_COLUMNS = ['round_status', 'map']
NUMERIC_COLUMNS = ['round_status_time_left']
INTEGER_COLUMNS = ['alive_players_t', 'alive_players_ct', "health_t", "health_ct", "money_ct", "money_t"]

def get_attr(ds, team, attr=None):
    team_players = map(lambda players: filter(lambda p: p["team"] == team, players), ds['alive_players'])
    if attr:
        team_players = map(lambda players: map(lambda p: p[attr], players), team_players)
    
    return list(map(lambda p: list(p), team_players))

for ds in [dftrain, dfeval]:
    ds['alive_players_t']  = list(map(len ,get_attr(ds, "Terrorist")))
    ds['alive_players_ct'] = list(map(len, get_attr(ds, "CT")))
    ds['health_ct']        = list(map(sum, get_attr(ds, "CT", "health")))
    ds['health_t']         = list(map(sum, get_attr(ds, "Terrorist", "health")))
    ds['money_ct']         = list(map(sum, get_attr(ds, "CT", "money")))
    ds['money_t']          = list(map(sum, get_attr(ds, "Terrorist", "money")))

print(dftrain.info())

feature_columns = []
feature_names = CATEGORICAL_COLUMNS + NUMERIC_COLUMNS + INTEGER_COLUMNS
for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float64))

for feature_name in INTEGER_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.int32))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71253 entries, 0 to 71252
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   map                     71253 non-null  object 
 1   patch_version           71253 non-null  int64  
 2   map_crc                 71253 non-null  int64  
 3   current_score           71253 non-null  object 
 4   round_status            71253 non-null  object 
 5   round_status_time_left  71253 non-null  float64
 6   alive_players           71253 non-null  object 
 7   active_smokes           71253 non-null  object 
 8   active_molotovs         71253 non-null  object 
 9   previous_kills          71253 non-null  object 
 10  planted_bomb            8034 non-null   object 
 11  alive_players_t         71253 non-null  int64  
 12  alive_players_ct        71253 non-null  int64  
 13  health_ct               71253 non-null  int64  
 14  health_t                71253 non-null

In [13]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df.filter(items=feature_names)), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function
        
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [14]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = linear_est.evaluate(eval_input_fn)
clear_output()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Vintel38\\AppData\\Local\\Temp\\tmp49q62ivr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
print(result)

In [ ]:
tf.keras.backend.set_floatx('float64')
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

In [ ]:
tf.get_logger().setLevel('WARN')

def predict_idx(data_df, idx):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices(dict(data_df.filter(items=feature_names)))
        ds = ds.skip(idx).take(1).batch(1)
        return ds
    return input_function

def predict_index(ds, idx):
    for feature in feature_names:
        print(feature, ds[feature][idx])
        
    print("")
    
    for pred in linear_est.predict(predict_idx(ds, idx)):
        probs = pred['probabilities']
        print("Win probability: CT: {:.2} T: {:.2}".format(probs[0], probs[1]))

    print("")

wt_names = linear_est.get_variable_names()
for name in wt_names:
    if name.endswith("weights"):
        val = linear_est.get_variable_value(name)
        print(name, val)

print(feature_columns)

for i in range(100,120):
    predict_index(dfeval, i)